To work on colab, we need to add some paths and install libraries that are already installed on local computer. This part is not needed to run on local machine.



In [ ]:
!pip install patchify
!pip install dropbox
!pip install gputil
!pip install psutil
!pip install humanize
!pip install mat73

     |████████████████████████████████| 582 kB 8.6 MB/s 
     |████████████████████████████████| 162 kB 59.8 MB/s 
     |████████████████████████████████| 49 kB 8.6 MB/s 
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=cb3bf8424455d53b87e76e79abd843b0773993be1280f3a3bced9ff7988e5c02
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil


We need to test the given GPU statistics

In [ ]:
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
# Execute function
mem_report()

CPU RAM Free: 26.2 GB
GPU 0 ... Mem Free: 16280MB / 16280MB | Utilization   0%


## Define Optional Settings

We need to set the necessary parameters for training. The path for the training and validation patches are automatically selected as local path or drive path.

batchSize: set the mini batch size for the training
patchSize: number of patches that are cropped from the training images
loadAllAtOnce: load all data to memory before running, this will increase the training process but memory could be insufficient

In [ ]:
# options
resolution = 20
patchSize = 300
patchOverlap = 116
batchSize = 16
patchCount = 200
numEpochs = 20
loadAllAtOnce = True
dataAugmentationTypes = ['original','hflip','vflip']
nband = 9
if resolution == 10:
  nband = 10

## Define Paths and Folders

Define the paths and folders for the training and testing.

In [ ]:
import os
dropboxFolderPath = 'E:/Dropbox/'

# define the paths relative to the dropbox folder
unetWorkingPath = os.path.join(dropboxFolderPath, 'Education/PhD/Projects/MucilageDetection/uNetLearning')

In [ ]:
import sys
from google.colab import drive

# mount the drive
drive.mount('/content/drive', force_remount=True)

# override the local paths
unetWorkingPath = '/content/drive/Othercomputers/My Laptop/uNetLearning/'

# add the paths to system paths 
sys.path.append(unetWorkingPath)
sys.path.append(os.path.join(unetWorkingPath, 'functions'))

Mounted at /content/drive


# Prepare For Train and Test

Training code has been written in python. We use uNet architecture to segment the mucilage from the water. The implementation fulfilled with pyTorch. *italicized text*

In [ ]:
# define common paths
modelSavePath = os.path.join(unetWorkingPath, 'models')
sentinelPatchImagePath = os.path.join(unetWorkingPath, 'patches' + str(resolution))

## Import Libraries

To utilize training, we need to import necessary libraries. Some of the libraries are the standart pyTorch libarries and these can be imported via colab. The custom libraries are imported via the google drive.

In [ ]:
# these are the standart libraries
import os
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchsummary import summary

# mat file and set imports
import json

# these are the custom libraries and will be imported from the drive
from unet.unet_model import UNet
from functions.ModelTrainer import train_model
from functions.SentinelLoader import SentinelPatchLoader
from functions.DataTransformer import GetDataTransformer

## Create Device

In [ ]:
# create pytorch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Training device is {device}')

# clear GPU memory
if device == 'cuda':
  torch.cuda.empty_cache()
  torch.cuda.clear_cache()

Training device is cuda


## Get DataTransformer for Training and Testing

We use the same transformer (data normalizer) for train and test. But it depends on the spatial resolution of the input.


In [ ]:
transformer = GetDataTransformer(resolution)

## Train and Validation Dataset

In [ ]:
# print the location of patches
if numEpochs > 0:
  print(f'Sentinel Patches will be used from {sentinelPatchImagePath}')

  # define train images
  TrainingImages = ["S2A_MSIL2A_20210509T084601_N0300_R107_T35TPF_20210509T115513",
                    "S2A_MSIL2A_20210512T085601_N0300_R007_T35TPF_20210512T133202",
                    "S2B_MSIL2A_20210514T084559_N0300_R107_T35TPF_20210514T113538"]
  TrainingDataLoader = SentinelPatchLoader(sentinelPatchImagePath, TrainingImages, patchCount, dataAugmentationTypes, loadAllAtOnce, transformer)

  # define validation images
  ValidationImages = ["S2A_MSIL2A_20210519T084601_N0300_R107_T35TPF_20210519T115101"]
  ValidationDataLoader = SentinelPatchLoader(sentinelPatchImagePath, ValidationImages, patchCount, dataAugmentationTypes, loadAllAtOnce, transformer)

Sentinel Patches will be used from /content/drive/Othercomputers/My Laptop/uNetLearning/patches20


In [ ]:
# create custom data loader
if numEpochs > 0:
  TrainingDataLoader = {
      'train': DataLoader(TrainingDataLoader, batch_size=batchSize, shuffle=True, num_workers=0),
      'val': DataLoader(ValidationDataLoader, batch_size=batchSize, shuffle=True, num_workers=0)
  }

## Create Model

Create the uNet model which has 9/10 channel input and 1 semantic class. After creating the model we will try to import the previous best weights for initialization. If the previous training run interrupted, code will try to recover from the last known state by loading the trainedModel. 

In [ ]:
# create a ResNetUNet model and apply it to model
if numEpochs > 0:
  trainModel = UNet(n_channels=nband, n_classes=1)
  trainModel = trainModel.to(device)

  # print the model summary
  summary(trainModel, (nband, patchSize, patchSize))

  # define optimizer and scheduler
  optimizer = optim.Adam(filter(lambda p: p.requires_grad, trainModel.parameters()), lr=0.0005)
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

  # check for previous model
  lastTrainedModelPath = os.path.join(modelSavePath, 'trainedModel' + str(resolution))
  preTrainedModelPath = os.path.join(modelSavePath,'bestModel' + str(resolution))
  if os.path.isfile(lastTrainedModelPath):
      print('loading the previous model...')
      
      # load the model
      checkpoint = torch.load(lastTrainedModelPath)
      trainModel.load_state_dict(checkpoint['model_state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      epoch = checkpoint['epoch']
      loss = checkpoint['loss']
  elif os.path.isfile(preTrainedModelPath):
      print('using the pre-trained model weights...')
      weights = torch.load(preTrainedModelPath)
      trainModel.load_state_dict(weights)
  else:
      print('no previous model found, training from scratch')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 300, 300]           5,248
       BatchNorm2d-2         [-1, 64, 300, 300]             128
              ReLU-3         [-1, 64, 300, 300]               0
            Conv2d-4         [-1, 64, 300, 300]          36,928
       BatchNorm2d-5         [-1, 64, 300, 300]             128
              ReLU-6         [-1, 64, 300, 300]               0
        DoubleConv-7         [-1, 64, 300, 300]               0
         MaxPool2d-8         [-1, 64, 150, 150]               0
            Conv2d-9        [-1, 128, 150, 150]          73,856
      BatchNorm2d-10        [-1, 128, 150, 150]             256
             ReLU-11        [-1, 128, 150, 150]               0
           Conv2d-12        [-1, 128, 150, 150]         147,584
      BatchNorm2d-13        [-1, 128, 150, 150]             256
             ReLU-14        [-1, 128, 1

# Training Code

Training code has been written in python. We use uNet architecture to segment the mucilage from the water. The implementation fulfilled with pyTorch.

## Define Train and Test Set

In this part we define **train** and **validation** set. Since we label only 4 images, we use 3 of them for training and 1 of them for validation. 

*Note that this part will take some time if the loadAllAtOnce flag set to True, but it will speed up the traing*

In [ ]:
# start training
if numEpochs > 0:
  print('training model...')
  train_model(trainModel, optimizer, exp_lr_scheduler, TrainingDataLoader, device, num_epochs=numEpochs, outputPath=modelSavePath)

training model...
Epoch 0/19
----------
LR 0.000125


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train: bce: 0.041184, dice: 0.194934, loss: 0.118059
val: bce: 0.056933, dice: 0.450134, loss: 0.253533
saving best model
1m 32s
Epoch 1/19
----------
LR 0.000125
train: bce: 0.035669, dice: 0.180452, loss: 0.108061
val: bce: 0.065600, dice: 0.492067, loss: 0.278833
1m 32s
Epoch 2/19
----------
LR 0.000125
train: bce: 0.035433, dice: 0.175526, loss: 0.105480
val: bce: 0.058326, dice: 0.461982, loss: 0.260154
1m 32s
Epoch 3/19
----------
LR 0.000125
train: bce: 0.034021, dice: 0.170441, loss: 0.102231
val: bce: 0.077164, dice: 0.518654, loss: 0.297909
1m 32s
Epoch 4/19
----------
LR 0.000125
train: bce: 0.033734, dice: 0.167239, loss: 0.100486
val: bce: 0.082265, dice: 0.528902, loss: 0.305583
1m 32s
Epoch 5/19
----------
LR 0.000125
train: bce: 0.033756, dice: 0.165062, loss: 0.099409
val: bce: 0.069368, dice: 0.449794, loss: 0.259581
1m 32s
Epoch 6/19
----------
LR 0.000125
train: bce: 0.032965, dice: 0.159879, loss: 0.096422
val: bce: 0.074240, dice: 0.487806, loss: 0.281023
1m 32s
E

# Test Model

In this section we are going to use the model trained in the previous section and generate the output for different images.

## Create Model and Load the Best Weights

Create the same model as in training phase and load the model parameters from the training.

In [ ]:
# include test related libraries
import numpy as np
import scipy.io as sio
import torch.nn.functional as func

from functions.SentinelLoader import SentinelTestDataset
from functions.TestImagePathFinder import GetTestImagePath
from ImagePatchHandler import ImagePatchHandler

## Create UNet model and load the weights

In [ ]:
testModel = UNet(n_channels=nband, n_classes=1)
testModel = testModel.to(device)
print('loading pretrained model...')
testModel.load_state_dict(torch.load(os.path.join(modelSavePath, 'bestModel' + str(resolution))))

loading pretrained model...


<All keys matched successfully>

## Create Output Directory

In [ ]:
# create output directory
outputDirectory = os.path.join(unetWorkingPath, "outputs")
if not os.path.exists(outputDirectory):
    os.makedirs(outputDirectory)

## Options

Set the test options

- modelResolution is needed for output array
- testbatchSize can be larger than the trainBatchSize
- cropZone is used to reduce tested patch size

In [ ]:
testBatchSize = 16

## Test Model on Images

In [ ]:
scaler = resolution / 10

# crop zone as x0,y0, width, height
cropZone = (0,0, int(8276 // scaler), int(3096 // scaler))
testImageFiles = 'testImages.json'
sentinelTestImagePath = '/Dataset/satellite/sentinel2/35TPE_MATDATA/'

# crop zone as x0,y0, width, height
# cropZone = (0,0, int(10980 // scaler), int(10980 // scaler))
# testImageFiles = 'trainImages.json'
# sentinelTestImagePath = '/Dataset/satellite/sentinel2/35TPF_MATDATA/'

jsonFile = open(os.path.join(unetWorkingPath, testImageFiles))
testImageFiles = json.load(jsonFile)
TestingImages = testImageFiles[str(resolution)]
jsonFile.close


# create patch handler
croppedImageSize = (cropZone[2], cropZone[3])
handler = ImagePatchHandler(croppedImageSize, (patchSize, patchSize), (patchOverlap, patchOverlap))

# test the images one by one and save the result
for TestImage in TestingImages:
    
    # get the path to the image
    ImagePath = GetTestImagePath(dropboxFolderPath, sentinelTestImagePath, TestImage)

    # create loader
    TestingDataLoader = SentinelTestDataset(ImagePath, cropZone, handler, transformer)
    
    # load the patches with batchSize
    TestDataLoader = {
        'test': DataLoader(TestingDataLoader, batch_size=testBatchSize, shuffle=False, num_workers=0)
    }
    
    # create output image
    result = np.zeros((croppedImageSize[1], croppedImageSize[0]), dtype=np.float32)
    
    # find the results
    for inputs, idx in TestDataLoader['test']:
        inputs = inputs.to(device)
    
        # get the result
        with torch.set_grad_enabled(False):
            outputs = func.sigmoid(testModel(inputs)).contiguous()
    
        # convert tensor to np array
        outputNP = outputs.cpu().detach().numpy()
        idxNP = idx.cpu().detach().numpy()

        # add the result into the big array
        for i in range(outputNP.shape[0]):
          result = handler.SetPatchImage(result, outputNP[i,:,:,:], idxNP[i])
        
    # convert patches to image
    OutputFileName = os.path.splitext(TestImage)[0]+'_MUCILAGE.mat'
    sio.savemat(os.path.join(outputDirectory, OutputFileName), {'mucilage': result})

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


## Test Model on Test Set

In [1]:
# define train images
TestingImages = ["S2A_MSIL2A_20210509T084601_N0300_R107_T35TPE_20210509T115513",
                  "S2B_MSIL2A_20210514T084559_N0300_R107_T35TPE_20210514T113538",
                  "S2B_MSIL2A_20210517T085559_N0300_R007_T35TPE_20210517T112912",
                  "S2A_MSIL2A_20210519T084601_N0300_R107_T35TPE_20210519T115101"]
TestingDataLoader = SentinelPatchLoader(sentinelPatchImagePath, TestingImages, patchCount, dataAugmentationTypes, loadAllAtOnce, transformer)

NameError: ignored

In [ ]:
# load the patches with batchSize
TestDataLoader = {
    'test': DataLoader(TestingDataLoader, batch_size=1, shuffle=False, num_workers=0)
}

# find the results
idx = 0
for inputs, labels in TestDataLoader['test']:
    inputs = inputs.to(device)

    # get the result
    with torch.set_grad_enabled(False):
        outputs = func.sigmoid(testModel(inputs)).contiguous()

    # get labels as contiguous array
    labelsC = labels.contiguous()

    # convert tensor to np array
    outputNP = outputs.cpu().detach().numpy()
    labelsNP = labelsC.cpu().detach().numpy()

    # save the results with the label
    OutputFileName = str(idx)+'_result.mat'
    sio.savemat(os.path.join(outputDirectory, OutputFileName), {'mucilage': outputNP, 'labels': labelsNP})

    # go to the next index
    idx = idx + 1